In [1]:
#!pip install tensorflow opencv-python matplotlib sklearn gTTS ipython

In [2]:
import tensorflow as tf
import numpy as np
import cv2
from gtts import gTTS
from IPython.display import Audio
import threading
import time
import enum

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches


In [3]:
# model = tf.keras.models.load_model('pushup-counter-softmax.h5')
# model = tf.keras.models.load_model('pushup-counter-with-reduced-landmark-and-sigmoid.h5')
# model = tf.keras.models.load_model('pushup-counter-movenet.h5')

In [4]:
pushup_model_folder = 'models/pushup-model-31may22.tflite'
pushup_model = tf.lite.Interpreter(model_path=pushup_model_folder)
pushup_model.allocate_tensors()

In [5]:
movenet_folder = 'models/movenet_lightning.tflite'
movenet = tf.lite.Interpreter(model_path=movenet_folder)
movenet.allocate_tensors()

In [6]:
# Source: the Movenet TFLite Documentation
class BodyPart(enum.Enum):
    """Enum representing human body keypoints detected by pose estimation models."""
    NOSE = 0
    LEFT_EYE = 1
    RIGHT_EYE = 2
    LEFT_EAR = 3
    RIGHT_EAR = 4
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6
    LEFT_ELBOW = 7
    RIGHT_ELBOW = 8
    LEFT_WRIST = 9
    RIGHT_WRIST = 10
    LEFT_HIP = 11
    RIGHT_HIP = 12
    LEFT_KNEE = 13
    RIGHT_KNEE = 14
    LEFT_ANKLE = 15
    RIGHT_ANKLE = 16

In [7]:
def processPose(image):
    img = tf.image.resize_with_pad(np.expand_dims(image, axis=0), 192,192)
    img = tf.cast(img, dtype=tf.float32)
    input_details = movenet.get_input_details()
    output_details = movenet.get_output_details()
    movenet.set_tensor(input_details[0]['index'], np.array(img))
    # Invoke inference.
    movenet.invoke()
    # Get the model prediction.
    keypoints_with_scores = movenet.get_tensor(output_details[0]['index'])
    keypoints = keypoints_with_scores.flatten().reshape(17,3)
    fixed_keypoints = []
    for [x, y, score] in keypoints:
        if score < 0.2:
            fixed_keypoints.append([0, 0, 0])
        else:
            fixed_keypoints.append([x, y, score])
    return np.array(fixed_keypoints)

In [8]:
def text_to_speech(sentence):
    tts = gTTS(sentence) #Provide the string to convert to speech
    sound_file = '1.wav'
    tts.save(sound_file) #save the string converted to speech as a .wav file
    display(Audio('1.wav', autoplay=True))

In [9]:
def body_is_visible(keypoints):
    keypoints = keypoints.reshape(17, 3)
    # shoulder, elbow, wrist, hip, knee must be visible
    important_bodyparts = [5, 7, 9, 11, 13] #this is only for the left part
    for index in important_bodyparts:
        if (keypoints[index][2] < 0.2) and (keypoints[index+1][2] < 0.2):  # one of the left and right bodypart needs to be seen
            return False
    return True

In [10]:
def is_pushup(keypoints):
    threshold = 0.9
    keypoints = tf.cast(keypoints, dtype=tf.float32) # convert from float64 to float32
    
    input_details = pushup_model.get_input_details()
    output_details = pushup_model.get_output_details()
    pushup_model.set_tensor(input_details[0]['index'], keypoints)
    # Invoke inference.
    pushup_model.invoke()
    # Get the model prediction.
    
    result = pushup_model.get_tensor(output_details[0]['index'])
    status = np.argmax(result)
    confidence = result[0][status]
#     print(result)
    
    if ((result[0][1] > threshold) or (result[0][2] > threshold)) and body_is_visible(np.array(keypoints)):
        return True
    return False
    
#     if (status == 1) and (confidence > threshold) and body_is_visible(np.array(keypoints)):
#         return True
#     return False

In [11]:
print(BodyPart.NOSE.value)

0


In [12]:
# im just trying to test with static image first
image = cv2.imread('008_139e64eb.jpg')
keypoints = processPose(image).flatten().reshape(1, 51)

print(is_pushup(keypoints))

True


In [13]:
def is_increasing(arr):
    # this list will show if the majority of the elements are increasing
    # from the previous element or not (True means it increased)
    status = []
    threshold = 0.8   # the minimum percentage of True status list element
    for i in range(1, len(arr)):
        if arr[i] > arr[i-1]:
            status.append(True)
        else:
            status.append(False)
    true_percentage = status.count(True) / (len(arr)-1)*1.0
    if true_percentage >= threshold:
        return True
    else:
        return False
    

def is_decreasing(arr):
    # this list will show if the majority of the elements are decreasing
    # from the previous element or not (True means it decreased)
    status = []
    threshold = 0.8   # the minimum percentage of True status list element
    for i in range(1, len(arr)):
        if arr[i] < arr[i-1]:
            status.append(True)
        else:
            status.append(False)
    true_percentage = status.count(True) / (len(arr)-1)*1.0
    if true_percentage >= threshold:
        return True
    else:
        return False

In [14]:
print(is_decreasing([329, 324, 322]))
range(1, len([329, 324, 322]))

True


range(1, 3)

In [15]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Middle
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [16]:
#@title Helper functions for visualization

# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Maps bones to a matplotlib color name.
KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
    """Returns high confidence keypoints and edges for visualization.

    Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

    Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
    """
    keypoints_all = []
    keypoint_edges_all = []
    edge_colors = []
    num_instances, _, _, _ = keypoints_with_scores.shape
    for idx in range(num_instances):
        kpts_x = keypoints_with_scores[0, idx, :, 1]
        kpts_y = keypoints_with_scores[0, idx, :, 0]
        kpts_scores = keypoints_with_scores[0, idx, :, 2]
        kpts_absolute_xy = np.stack(
            [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
        kpts_above_thresh_absolute = kpts_absolute_xy[
            kpts_scores > keypoint_threshold, :]
        keypoints_all.append(kpts_above_thresh_absolute)

        for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
            if (kpts_scores[edge_pair[0]] > keypoint_threshold and
              kpts_scores[edge_pair[1]] > keypoint_threshold):
                x_start = kpts_absolute_xy[edge_pair[0], 0]
                y_start = kpts_absolute_xy[edge_pair[0], 1]
                x_end = kpts_absolute_xy[edge_pair[1], 0]
                y_end = kpts_absolute_xy[edge_pair[1], 1]
                line_seg = np.array([[x_start, y_start], [x_end, y_end]])
                keypoint_edges_all.append(line_seg)
                edge_colors.append(color)
    if keypoints_all:
        keypoints_xy = np.concatenate(keypoints_all, axis=0)
    else:
        keypoints_xy = np.zeros((0, 17, 2))

    if keypoint_edges_all:
        edges_xy = np.stack(keypoint_edges_all, axis=0)
    else:
        edges_xy = np.zeros((0, 2, 2))
    return keypoints_xy, edges_xy, edge_colors


def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
    """Draws the keypoint predictions on image.

    Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

    Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
    """
    height, width, channel = image.shape
    aspect_ratio = float(width) / height
    fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
    # To remove the huge white borders
    fig.tight_layout(pad=0)
    ax.margins(0)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.axis('off')

    im = ax.imshow(image)
    line_segments = LineCollection([], linewidths=(4), linestyle='solid')
    ax.add_collection(line_segments)
    # Turn off tick labels
    scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

    (keypoint_locs, keypoint_edges,
    edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
    if keypoint_edges.shape[0]:
        line_segments.set_segments(keypoint_edges)
        line_segments.set_color(edge_colors)
    if keypoint_locs.shape[0]:
        scat.set_offsets(keypoint_locs)

    if crop_region is not None:
        xmin = max(crop_region['x_min'] * width, 0.0)
        ymin = max(crop_region['y_min'] * height, 0.0)
        rec_width = min(crop_region['x_max'], 0.99) * width - xmin
        rec_height = min(crop_region['y_max'], 0.99) * height - ymin
        rect = patches.Rectangle(
            (xmin,ymin),rec_width,rec_height,
            linewidth=1,edgecolor='b',facecolor='none')
        ax.add_patch(rect)

    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    if output_image_height is not None:
        output_image_width = int(output_image_height / height * width)
        image_from_plot = cv2.resize(
            image_from_plot, dsize=(output_image_width, output_image_height),
             interpolation=cv2.INTER_CUBIC)
    return image_from_plot

In [17]:
 def movenet_process(input_image):
    input_image = tf.image.resize_with_pad(np.expand_dims(input_image, axis=0), 192,192)
    input_image = tf.cast(input_image, dtype=tf.float32)
    input_details = movenet.get_input_details()
    output_details = movenet.get_output_details()
    movenet.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    movenet.invoke()
    # Get the model prediction.
    keypoints_with_scores = movenet.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

In [22]:
shoulder_height = 0
history = []
fail_count = 0
pushup_count = 0
num_frames_requirement = 5
pushup_down_done = False

cap = cv2.VideoCapture("tests/pushup-video-1x.mp4")

while cap.isOpened():
    # Read feed
    ret, frame = cap.read()
    #frame = cv2.flip(frame, 1)
    
    if ret != True:
        break

    cv2.putText(frame, "push up count:"+str(pushup_count), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    keypoints = processPose(frame).flatten().reshape(1, 51)
    
    if is_pushup(keypoints):
        print("*", end ="")
        fail_count = 0
        keypoints = keypoints.reshape(17, 3)
        
        if keypoints[BodyPart.NOSE.value][1] > keypoints[BodyPart.RIGHT_KNEE.value][1]: #bagian kanan di depan kamera
            shoulder_y = keypoints[BodyPart.RIGHT_SHOULDER.value][0] # the y coordinate is in the first element
        else:
            shoulder_y = keypoints[BodyPart.LEFT_SHOULDER.value][0]     

        head_height = shoulder_y 
        
        head_height = int(head_height*100)
        
        if (len(history) == 0) or (head_height != history[-1]):
            history.append(head_height)
            print(head_height, end=" ")
            
            if len(history) >= num_frames_requirement:
                print(history[-num_frames_requirement:], end=" ")
                if is_decreasing(history[-num_frames_requirement:]):
                    pushup_down_done = True
                    print("dw", end=" ")
                elif is_increasing(history[-num_frames_requirement:]):
                    print("up", end=" ")
                    if pushup_down_done:
                        pushup_count += 1
                        print(pushup_count)
                        pushup_down_done = False
                        cv2.putText(frame, "push up count:"+str(pushup_count), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                        threading.Thread(target=text_to_speech, args=[str(pushup_count)+" pushup"]).start()
        
#     else:
#         print("-", end ="")
#         fail_count += 1
#         if fail_count > 100:
#             history = []
#             pushup_down_done = False

    cv2.imshow('Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

*46 ***48 *45 ***46 *****47 [46, 48, 45, 46, 47] *46 [48, 45, 46, 47, 46] *47 [45, 46, 47, 46, 47] *46 [46, 47, 46, 47, 46] ***45 [47, 46, 47, 46, 45] *******46 [46, 47, 46, 45, 46] **********47 [47, 46, 45, 46, 47] *****48 [46, 45, 46, 47, 48] *51 [45, 46, 47, 48, 51] up *55 [46, 47, 48, 51, 55] up *56 [47, 48, 51, 55, 56] up *60 [48, 51, 55, 56, 60] up *61 [51, 55, 56, 60, 61] up *62 [55, 56, 60, 61, 62] up *64 [56, 60, 61, 62, 64] up ****63 [60, 61, 62, 64, 63] *62 [61, 62, 64, 63, 62] **60 [62, 64, 63, 62, 60] *59 [64, 63, 62, 60, 59] dw *57 [63, 62, 60, 59, 57] dw *55 [62, 60, 59, 57, 55] dw *50 [60, 59, 57, 55, 50] dw *49 [59, 57, 55, 50, 49] dw *47 [57, 55, 50, 49, 47] dw *46 [55, 50, 49, 47, 46] dw **47 [50, 49, 47, 46, 47] **49 [49, 47, 46, 47, 49] *50 [47, 46, 47, 49, 50] *52 [46, 47, 49, 50, 52] up 1
*54 [47, 49, 50, 52, 54] up *56 [49, 50, 52, 54, 56] up *58 [50, 52, 54, 56, 58] up *59 [52, 54, 56, 58, 59] up 

*60 [54, 56, 58, 59, 60] up *61 [56, 58, 59, 60, 61] up *62 [58, 59, 60, 61, 62] up *64 [59, 60, 61, 62, 64] up ****62 [60, 61, 62, 64, 62] **61 [61, 62, 64, 62, 61] *60 [62, 64, 62, 61, 60] *57 [64, 62, 61, 60, 57] dw *56 [62, 61, 60, 57, 56] dw *54 [61, 60, 57, 56, 54] dw *51 [60, 57, 56, 54, 51] dw *49 [57, 56, 54, 51, 49] dw *50 [56, 54, 51, 49, 50] *52 [54, 51, 49, 50, 52] *54 [51, 49, 50, 52, 54] *56 [49, 50, 52, 54, 56] up 2
*59 [50, 52, 54, 56, 59] up *60 [52, 54, 56, 59, 60] up 

*62 [54, 56, 59, 60, 62] up *64 [56, 59, 60, 62, 64] up ***63 [59, 60, 62, 64, 63] *62 [60, 62, 64, 63, 62] *61 [62, 64, 63, 62, 61] *60 [64, 63, 62, 61, 60] dw *58 [63, 62, 61, 60, 58] dw *57 [62, 61, 60, 58, 57] dw *55 [61, 60, 58, 57, 55] dw *52 [60, 58, 57, 55, 52] dw *50 [58, 57, 55, 52, 50] dw *49 [57, 55, 52, 50, 49] dw *48 [55, 52, 50, 49, 48] dw ***49 [52, 50, 49, 48, 49] *50 [50, 49, 48, 49, 50] *55 [49, 48, 49, 50, 55] *58 [48, 49, 50, 55, 58] up 3
*59 [49, 50, 55, 58, 59] up *60 [50, 55, 58, 59, 60] up 

*61 [55, 58, 59, 60, 61] up *62 [58, 59, 60, 61, 62] up *64 [59, 60, 61, 62, 64] up ***62 [60, 61, 62, 64, 62] **59 [61, 62, 64, 62, 59] *58 [62, 64, 62, 59, 58] *56 [64, 62, 59, 58, 56] dw *51 [62, 59, 58, 56, 51] dw *49 [59, 58, 56, 51, 49] dw *48 [58, 56, 51, 49, 48] dw *47 [56, 51, 49, 48, 47] dw ***48 [51, 49, 48, 47, 48] *50 [49, 48, 47, 48, 50] *55 [48, 47, 48, 50, 55] *60 [47, 48, 50, 55, 60] up 4
*62 [48, 50, 55, 60, 62] up 

*63 [50, 55, 60, 62, 63] up *64 [55, 60, 62, 63, 64] up 

In [18]:
# shoulder_height = 0
# history = []
# fail_count = 0
# pushup_count = 0
# num_frames_requirement = 5
# pushup_down_done = False


# cap = cv2.VideoCapture("tests/pushup-video-1x.mp4")

# while cap.isOpened():
#     # Read feed
#     ret, frame = cap.read()
#     #frame = cv2.flip(frame, 1)
    
#     if ret != True:
#         break
#     status = False
# #     cv2.putText(frame, "push up count:"+str(pushup_count), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#     keypoints = processPose(frame).flatten().reshape(1, 51)
    
#     if is_pushup(keypoints):
# #         print("*", end ="")
# #         fail_count = 0
#         status = True
#         keypoints = keypoints.reshape(17, 3)
# #         print(keypoints[BodyPart.NOSE.value][1], end=" ")
# #         print(keypoints[BodyPart.RIGHT_KNEE.value][1])
        
#         if keypoints[BodyPart.NOSE.value][1] < keypoints[BodyPart.RIGHT_KNEE.value][1]: #badan bagian kanan ke arah kamera
#             knee_xy = keypoints[BodyPart.RIGHT_KNEE.value][:2]
#             shoulder_xy = keypoints[BodyPart.RIGHT_SHOULDER.value][:2]
#             wrist_xy = keypoints[BodyPart.RIGHT_WRIST.value][:2]
#             elbow_xy = keypoints[BodyPart.RIGHT_ELBOW.value][:2]
#         else:
#             knee_xy = keypoints[BodyPart.LEFT_KNEE.value][:2]
#             shoulder_xy = keypoints[BodyPart.LEFT_SHOULDER.value][:2]
#             wrist_xy = keypoints[BodyPart.LEFT_WRIST.value][:2]   
#             elbow_xy = keypoints[BodyPart.LEFT_ELBOW.value][:2]

#         angle = calculate_angle(shoulder_xy, elbow_xy, wrist_xy)
#         cv2.putText(frame, "angle:"+str(int(angle)), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
# #         print(angle)
        
# #         if (len(history) == 0) or (head_height != history[-1]):
# #             history.append(head_height)
# #             print(head_height, end=" ")
            
# #             if len(history) >= num_frames_requirement:
# #                 print(history[-num_frames_requirement:], end=" ")
# #                 if is_decreasing(history[-num_frames_requirement:]):
# #                     pushup_down_done = True
# #                     print("dw", end=" ")
# #                 elif is_increasing(history[-num_frames_requirement:]):
# #                     print("up", end=" ")
# #                     if pushup_down_done:
# #                         pushup_count += 1
# #                         print(pushup_count)
# #                         pushup_down_done = False
# #                         cv2.putText(frame, "push up count:"+str(pushup_count), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
# #                         threading.Thread(target=text_to_speech, args=[str(pushup_count)+" pushup"]).start()
        
# #     else:
# #         print("-", end ="")
# #         fail_count += 1
# #         if fail_count > 100:
# #             history = []
# #             pushup_down_done = False

#     output = draw_prediction_on_image(frame, movenet_process(frame))
#     cv2.imshow('Frame', output)
# #     if(status):
# #         time.sleep(0.5)
    
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break
# cap.release()
# cv2.destroyAllWindows()